In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install plotly
!pip install cufflinks
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install keras
!pip install nltk
!pip install wordcloud
!pip install regex


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import cufflinks as cf
import transformers
import torch
import tensorflow as tf
import keras
import nltk
import wordcloud
import regex
import nltk

















In [ ]:
data = pd.read_excel('/content/talha737 (1).xlsx')

In [ ]:
# Dropping rows where 'description' contains NaN or an empty string
data = data.dropna(subset=['description'])
data


,id,Product_id,Product_flavor_id,Shopper_id,User_id,Orderitem_id,description,images,video,rating,rating_reason,reply,status,date_create,date_modify
0,103938,8391,19091,63990000000000000,1000029,1536548,Good experience myself fast time purchase onli...,[],NaN,4,Customer Service,NaN,1,2024-09-14 04:01:00,2024-09-14 04:05:00
1,103939,7040,14698,5529390000000000000,1000029,1422599,Excellent product ��������,[],NaN,5,Product Quality,NaN,1,2024-09-14 04:45:00,2024-09-14 05:05:00
2,103940,4664,9943,388724000000000000,1000029,1537066,Good watch in this price. But if u wnt to buy ...,[],NaN,3,Other,NaN,0,2024-09-14 06:01:00,2024-09-14 06:05:00
3,103944,7506,15829,3470420000000000000,1000234,1548132,Fully stasfied from services,[],NaN,5,Customer Service,NaN,1,2024-09-14 08:17:00,2024-09-14 17:48:00
4,103945,7471,16734,350205000000000000,1000234,1457371,Used product is packed _x000D_\nso be careful ...,[],NaN,1,Other,NaN,0,2024-09-14 09:08:00,2024-09-15 11:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,105695,8305,17641,4000400000000000000,1000234,1573693,"packing is bad, priceoye box and device box bo...","[{image"": ""1348769-ug0ej.jpg""","""thumbnails"": {""94x94"": ""1348769-ug0ej-94x94....","""270x270"": ""1348769-ug0ej-270x270.jpg""}}","{""image"": ""1348769-s4h43.jpg""","""thumbnails"": {""94x94"": ""1348769-s4h43-94x94....","""270x270"": ""1348769-s4h43-270x270.jpg""}}]""",NaN,4
1346,105696,7407,15622,3591320000000000000,1000234,1561841,Mashallah Koi ishu Nahin hua Jo chij mangvaee ...,[],NaN,5,Product Price,NaN,1,2024-09-27 21:58:00,2024-09-28 14:39:00
1347,105697,8006,16954,4021680000000000000,1000234,1561775,Best service by priceoye,[],NaN,5,Other,NaN,1,2024-09-27 22:41:00,2024-09-28 14:39:00
1348,105698,7391,15589,1989590000000000000,1000234,1215226,Excellent service fully satisfied,[],NaN,5,Packaging,NaN,1,2024-09-27 22:53:00,2024-09-28 14:39:00


In [ ]:
from transformers import pipeline
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# Load Sentiment Analysis Model
sentiment_classifier = pipeline("sentiment-analysis", model="akhooli/xlm-r-large-arabic-sent", device=-1)

# Load Zero-Shot Classification Models
classifiers = {
    "deberta": pipeline(task="zero-shot-classification", model="microsoft/deberta-v3-large", device=-1),
    "mbart": pipeline(task="zero-shot-classification", model="facebook/mbart-large-50", device=-1),
    "xlm": pipeline(task="zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=-1)
}

# Define your custom labels based on sentiment
positive_labels = ["Positive Feedback", "Product Recommendations"]
negative_labels = ["Product Quality Issue", "Authenticity and Originality Concerns", "Delivery and Logistics Issues",
                   "Customer Service Experience", "Returns, Refunds, and Exchange Issues", "Scam Complaint", "Pricing and Billing Issues","Warranty and Support Complaints"]
neutral_labels = positive_labels + negative_labels

# Function to perform sentiment analysis
def get_sentiment(text):
    sentiment = sentiment_classifier(text)
    return sentiment[0]['label'].lower()

# Function to classify text based on sentiment
def classify_based_on_sentiment(text, sentiment, classifiers, labels_map):
    if sentiment == "positive":
        candidate_labels = positive_labels
    elif sentiment == "negative":
        candidate_labels = negative_labels
    else:
        candidate_labels = neutral_labels

    # Zero-shot classification using filtered labels
    results = {}
    for model_name, classifier in classifiers.items():
        predictions = classifier(text, candidate_labels, multi_class=True)
        results[model_name] = predictions

    # Aggregate and select the best label based on averaged scores
    label_scores = {}
    for model_name, predictions in results.items():
        for label, score in zip(predictions['labels'], predictions['scores']):
            if label in label_scores:
                label_scores[label].append(score)
            else:
                label_scores[label] = [score]

    # Calculate the average top 2 scores per label
    final_scores = {label: sum(sorted(scores, reverse=True)[:2]) / 2 for label, scores in label_scores.items()}
    best_label = max(final_scores, key=final_scores.get)

    return best_label



# Ensure descriptions are strings
data['description'] = data['description'].astype(str)

# Placeholder for predicted labels
data['Review_CAT'] = None

# Process each description
for idx, row in data.iterrows():
    text = row['description']
    sentiment = get_sentiment(text)  # Step 1: Perform sentiment analysis
    label = classify_based_on_sentiment(text, sentiment, classifiers, labels_map=None)  # Step 2: Zero-shot classification within filtered labels
    data.at[idx, 'Review_CAT'] = label

# Output the results
print(data)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transform

KeyboardInterrupt: 

In [ ]:
data

,id,Product_id,Product_flavor_id,Shopper_id,User_id,Orderitem_id,description,images,video,rating,rating_reason,reply,status,date_create,date_modify,Review_CAT
0,103938,8391,19091,63990000000000000,1000029,1536548,Good experience myself fast time purchase onli...,[],NaN,4,Customer Service,NaN,1,2024-09-14 04:01:00,2024-09-14 04:05:00,Positive Feedback
1,103939,7040,14698,5529390000000000000,1000029,1422599,Excellent product ��������,[],NaN,5,Product Quality,NaN,1,2024-09-14 04:45:00,2024-09-14 05:05:00,Positive Feedback
2,103940,4664,9943,388724000000000000,1000029,1537066,Good watch in this price. But if u wnt to buy ...,[],NaN,3,Other,NaN,0,2024-09-14 06:01:00,2024-09-14 06:05:00,Product Quality Issue
3,103944,7506,15829,3470420000000000000,1000234,1548132,Fully stasfied from services,[],NaN,5,Customer Service,NaN,1,2024-09-14 08:17:00,2024-09-14 17:48:00,Positive Feedback
4,103945,7471,16734,350205000000000000,1000234,1457371,Used product is packed _x000D_\nso be careful ...,[],NaN,1,Other,NaN,0,2024-09-14 09:08:00,2024-09-15 11:59:00,Product Quality Issue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,105695,8305,17641,4000400000000000000,1000234,1573693,"packing is bad, priceoye box and device box bo...","[{image"": ""1348769-ug0ej.jpg""","""thumbnails"": {""94x94"": ""1348769-ug0ej-94x94....","""270x270"": ""1348769-ug0ej-270x270.jpg""}}","{""image"": ""1348769-s4h43.jpg""","""thumbnails"": {""94x94"": ""1348769-s4h43-94x94....","""270x270"": ""1348769-s4h43-270x270.jpg""}}]""",NaN,4,None
1346,105696,7407,15622,3591320000000000000,1000234,1561841,Mashallah Koi ishu Nahin hua Jo chij mangvaee ...,[],NaN,5,Product Price,NaN,1,2024-09-27 21:58:00,2024-09-28 14:39:00,None
1347,105697,8006,16954,4021680000000000000,1000234,1561775,Best service by priceoye,[],NaN,5,Other,NaN,1,2024-09-27 22:41:00,2024-09-28 14:39:00,None
1348,105698,7391,15589,1989590000000000000,1000234,1215226,Excellent service fully satisfied,[],NaN,5,Packaging,NaN,1,2024-09-27 22:53:00,2024-09-28 14:39:00,None


In [ ]:
data.to_excel('talha737_output.xlsx', index = False)